In [1]:
import numpy as np
import xarray as xr 
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cf
import glob
import dask.array as da
import time
import seaborn as sns
import pandas as pd
import matplotlib.path as mpath
# colormaps
import cmaps
from cmcrameri import cm
import cmasher as cmr
import cmocean
import gc

In [57]:
from dask.distributed import Client
client = Client(scheduler_file= '/home/huanglei/proj_pyncl/CESM2-LENS/scheduler.json')
client

<Client: 'tcp://203.247.189.224:41018' processes=16 threads=576, memory=298.02 GiB>

In [4]:
def def_process_coords(exceptcv=[]):
    def process_coords(ds, except_coord_vars=exceptcv):
        coord_vars = []
        for v in np.array(ds.coords):
            if not v in except_coord_vars:
                coord_vars += [v]
        for v in np.array(ds.data_vars):
            if not v in except_coord_vars:
                coord_vars += [v]
        return ds.drop(coord_vars)
    return process_coords

In [7]:
## define function to read in files
def read_in(var, exceptcv, domain='lnd/', freq='day_1/', stream='h6', chunks=dict(time=365), ens_s=-20, ens_e=-10):
    ens_dir = "the_path_to_yourdirectory_store_data"
    histens_names = [member.split('archive/')[1][:-1]
                     for member in sorted(glob.glob(ens_dir + "b.e21.BHIST*LE2*[!old][!tmp]/"))][10:]
    projens_names = [member.split('archive/')[1][:-1] for member in sorted(
        glob.glob(ens_dir + "b.e21.BSSP370*.f09_g17*[!old][!tmp]/"))][10:]
    hist_ncfiles = []
    proj_ncfiles = []
    for i in np.arange(ens_s, ens_e):
        hist_fnames = sorted(glob.glob(
            ens_dir + histens_names[i] + "/" + domain + "proc/tseries/" + freq + "*" + stream + var + "*"))
        proj_fnames = sorted(glob.glob(
            ens_dir + projens_names[i] + "/" + domain + "proc/tseries/" + freq + "*" + stream + var + "*"))
        hist_ncfiles.append(hist_fnames)
        proj_ncfiles.append(proj_fnames)
    ens_numbers = [members.split('LE2-')[1]
                   for members in histens_names][ens_s:ens_e]
    hist_ds = xr.open_mfdataset(hist_ncfiles,
                                chunks=chunks,
                                preprocess=def_process_coords(exceptcv),
                                combine='nested',
                                concat_dim=[[*ens_numbers], 'time'],
                                parallel=True)
    proj_ds = xr.open_mfdataset(proj_ncfiles,
                                chunks=chunks,
                                preprocess=def_process_coords(exceptcv),
                                combine='nested',
                                concat_dim=[[*ens_numbers], 'time'],
                                parallel=True)
    if freq == 'day_1/':
        hist_ds = hist_ds.isel(time=np.arange(1, hist_ds.time.shape[0]))
        proj_ds = proj_ds.isel(time=np.arange(1, proj_ds.time.shape[0]))
        hist_ds['time'] = hist_ds.time.get_index('time').shift(-1, 'D')
        proj_ds['time'] = proj_ds.time.get_index('time').shift(-1, 'D')
    if freq == 'month_1/':
        hist_ds['time'] = hist_ds.time.get_index('time').shift(-1, 'M')
        proj_ds['time'] = proj_ds.time.get_index('time').shift(-1, 'M')
    ens_ds = xr.concat((hist_ds, proj_ds), 'time')
    ens_ds = ens_ds.rename({'concat_dim': 'ensemble'})
    return ens_ds


In [28]:
lake_ixy = xr.open_dataarray('/your_path/lake_ixy.nc')
lake_jxy = xr.open_dataarray('/your_path/lake_jxy.nc')


# read in data and calculate the phenology

In [8]:
variables = ['TLAKE']
exceptcv = ['time', 'levlak','lat', 'lon', *variables]
tlake_day_ds = read_in(var = '.TLAKE.', 
                    exceptcv = exceptcv,
                     domain = 'lnd/',
                     freq = 'day_1/',
                     stream = 'h5',
                    ens_s = 0,
                    ens_e = 90) # 90 out of 100 members have daily data that allows our analysis

variables = ['LAKEICETHICK']
exceptcv = ['time', 'lat', 'lon', *variables]
ice_day_ds = read_in(var = '.LAKEICETHICK.', 
                    exceptcv = exceptcv,
                     domain = 'lnd/',
                     freq = 'day_1/',
                     stream = 'h5',
                    ens_s = 0,
                    ens_e = 90)

variables = ['JCONVECT']
exceptcv = ['time', 'lat', 'lon', *variables]
jconv_day_ds = read_in(var = '.JCONVECT.', 
                    exceptcv = exceptcv,
                     domain = 'lnd/',
                     freq = 'day_1/',
                     stream = 'h5',
                    ens_s = 0,
                    ens_e = 90)

variables = ['U10']
exceptcv = ['time', 'lat','lon', *variables]
u10_day_ds = read_in(var = '.U10.', 
                    exceptcv = exceptcv,
                     domain = 'atm/',
                     freq = 'day_1/',
                     stream = 'h1',
                       ens_s = 0,
                       ens_e = 90
                      )

/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/xarray/core/indexing.py:1440: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [33]:
# this is the read_in dataset look like (taking TLAKE as an example), you may want to read the original data from CESM2-LE output following the same 
# method, to make sure the dimension of arrays be the same, so that you can use this script to reproduce all the analysis
tlake_day_ds

<xarray.Dataset>
Dimensions:   (levlak: 10, lon: 288, lat: 192, time: 91615, ensemble: 90)
Coordinates:
  * levlak    (levlak) float32 0.05 0.6 2.1 4.6 8.1 12.6 18.6 25.6 34.33 44.78
  * lon       (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat       (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * time      (time) object 1850-01-01 00:00:00 ... 2100-12-31 00:00:00
  * ensemble  (ensemble) object '1231.001' '1231.002' ... '1301.019' '1301.020'
Data variables:
    TLAKE     (ensemble, time, levlak, lat, lon) float32 dask.array<chunksize=(1, 364, 10, 192, 288), meta=np.ndarray>
Attributes: (12/100)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   created on 03/13/20 13:57:49
    source:                                    Community Land Model CLM4.0
    hostname:                                  aleph
    ...                                        ...
    cft_irrigated_switchgrass:                 60
    cft_tropical_corn:                         61
    cft_irrigated_tropical_corn:               62
    cft_tropical_soybean:                      63
    cft_irrigated_tropical_soybean:            64
    time_period_freq:                          day_1

In [18]:
'''since we only care about ice covred lakes, thus build the mask for ice cover first'''
ice_ann_climmean_1850s = (ice_day_ds.LAKEICETHICK[-20:,:3650,:,:] > 0).groupby('time.year').sum('time', skipna=True).mean('year').mean('ensemble').compute().where(~np.isnan(ice_day_ds.LAKEICETHICK[0,0,...].drop(('ensemble','time')).compute()))
ice_ann_climmean_1850s_lunit = ice_ann_climmean_1850s[lake_jxy,lake_ixy]

In [19]:
# read in global mean surface air temperature over land

trefht_globmean = pd.read_csv('/your_path_to_store_data/trefht_glob_mean.csv')

# calculate the climatological mean of stratification

In [23]:
strat_days_1850s_lunit = (xr.apply_ufunc(lambda x: x.max() - x[x > 0].min(),
              tlake_day_ds.TLAKE[:,:3650,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'time':30,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.year').sum('time').mean(('year','ensemble')).compute()
inverstrat_days_1850s_lunit = (xr.apply_ufunc(lambda x: x[-1] - x[x>0][0],
              tlake_day_ds.TLAKE[:,:3650,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'time':30,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.year').sum('time').mean(('year','ensemble')).compute()
positstrat_days_1850s_lunit = (xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:3650,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'time':30,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.year').sum('time').mean(('year','ensemble')).compute()

/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 63.97 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 64.02 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 63.98 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [34]:
strat_days_mon_1850s_lunit = ((xr.apply_ufunc(lambda x: x.max() - x[x > 0].min(),
              tlake_day_ds.TLAKE[:,:3650,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time') / 10).mean('ensemble').compute()
inverstrat_days_mon_1850s_lunit = ((xr.apply_ufunc(lambda x: x[-1] - x[x>0][0],
              tlake_day_ds.TLAKE[:,:3650,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time') / 10).mean('ensemble').compute()
positstrat_days_mon_1850s_lunit = ((xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:3650,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time') / 10).mean('ensemble').compute()

Task exception was never retrieved
future: <Task finished name='Task-26588' coro=<Client._gather.<locals>.wait() done, defined at /home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:2187> exception=AllExit()>
Traceback (most recent call last):
  File "/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py", line 2196, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-26589' coro=<Client._gather.<locals>.wait() done, defined at /home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:2187> exception=AllExit()>
Traceback (most recent call last):
  File "/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py", line 2196, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-26590' coro=<Client._gather

/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 49.81 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 49.87 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [35]:
strat_days_1850s_ensmean = np.empty((192,288)) * np.nan
for i in np.arange(4622):
    strat_days_1850s_ensmean[lake_jxy[i], lake_ixy[i]] = strat_days_1850s_lunit[i].values
strat_days_1850s_ensmean = xr.DataArray(
    strat_days_1850s_ensmean,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'annual duration of stratification (sum of summer and inverse stratification)',
           'units': 'days'},
    name='STRAT'
)

inverstrat_days_1850s_ensmean = np.empty((192,288)) * np.nan
for i in np.arange(4622):
    inverstrat_days_1850s_ensmean[lake_jxy[i], lake_ixy[i]] = inverstrat_days_1850s_lunit[i].values
inverstrat_days_1850s_ensmean = xr.DataArray(
    inverstrat_days_1850s_ensmean,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'annual duration of inverse stratification',
           'units': 'days'},
    name='INVER_STRAT'
)

positstrat_days_1850s_ensmean = np.empty((192,288)) * np.nan
for i in np.arange(4622):
    positstrat_days_1850s_ensmean[lake_jxy[i], lake_ixy[i]] = positstrat_days_1850s_lunit[i].values
positstrat_days_1850s_ensmean = xr.DataArray(
    positstrat_days_1850s_ensmean,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'annual duration of summer stratification',
           'units': 'days'},
    name='SUMMER_STRAT'
)

overturn_jconv_days_1850s = (jconv_day_ds.JCONVECT[:,:3650,:,:] > 8).groupby('time.year').sum('time').mean(('year','ensemble')).compute().where(~np.isnan(strat_days_1850s_ensmean))

In [36]:
strat_days_1850s_mon_ensmean = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    strat_days_1850s_mon_ensmean[:,lake_jxy[i], lake_ixy[i]] = strat_days_mon_1850s_lunit[:,i].values
strat_days_1850s_mon_ensmean = xr.DataArray(
    strat_days_1850s_mon_ensmean,
    dims=('month','lat', 'lon'),
    coords={'month':np.arange(1,13),'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'monthly duration of stratification (sum of summer and inverse stratification)',
           'units': 'days'},
    name='STRAT'
)

inverstrat_days_1850s_mon_ensmean = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    inverstrat_days_1850s_mon_ensmean[:,lake_jxy[i], lake_ixy[i]] = inverstrat_days_mon_1850s_lunit[:,i].values
inverstrat_days_1850s_mon_ensmean = xr.DataArray(
    inverstrat_days_1850s_mon_ensmean,
    dims=('month','lat', 'lon'),
    coords={'month':np.arange(1,13),'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'monthly duration of inverse stratification',
           'units': 'days'},
    name='INVER_STRAT'
)

positstrat_days_1850s_mon_ensmean = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    positstrat_days_1850s_mon_ensmean[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_mon_1850s_lunit[:,i].values
positstrat_days_1850s_mon_ensmean = xr.DataArray(
    positstrat_days_1850s_mon_ensmean,
    dims=('month','lat', 'lon'),
    coords={'month':np.arange(1,13),'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'monthly duration of summer stratification',
           'units': 'days'},
    name='SUMMER_STRAT'
)

overturn_jconv_days_mon_1850s = ((jconv_day_ds.JCONVECT[:,:3650,:,:] > 8).groupby('time.month').sum('time') / 10).mean('ensemble').compute().where(~np.isnan(strat_days_1850s_ensmean))

In [ ]:

tsa_mon_climmean_1850s_lunit = tsa_mon_ds.TSA[:, :120,lake_jxy,lake_ixy].groupby('time.month').mean('time').mean('ensemble').compute() - 273.15

tsa_mon_climmean_1850s = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    tsa_mon_climmean_1850s[:,lake_jxy[i], lake_ixy[i]] = tsa_mon_climmean_1850s_lunit[:,i].values
tsa_mon_climmean_1850s = xr.DataArray(
    tsa_mon_climmean_1850s,
    dims=('month','lat', 'lon'),
    coords={'month':np.arange(1,13),'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'climatological mean of monthly surface air temperature)',
           'units': 'days'},
    name='TSA'
)

tsa_mon_climmean_2090s_lunit = tsa_mon_ds.TSA[:, -120:,lake_jxy,lake_ixy].groupby('time.month').mean('time').mean('ensemble').compute() - 273.15

tsa_mon_climmean_2090s = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    tsa_mon_climmean_2090s[:,lake_jxy[i], lake_ixy[i]] = tsa_mon_climmean_2090s_lunit[:,i].values
tsa_mon_climmean_2090s = xr.DataArray(
    tsa_mon_climmean_2090s,
    dims=('month','lat', 'lon'),
    coords={'month':np.arange(1,13),'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'climatological mean of monthly surface air temperature)',
           'units': 'days'},
    name='TSA'
)

In [39]:
overturn_jconv_mon_1850s = ((jconv_day_ds.JCONVECT[:, :3650,:,:] > 8 ).groupby('time.month').sum('time').mean('ensemble') / 10).compute().where(~np.isnan(strat_days_1850s_ensmean))

In [131]:
ice_mon_climmean_1850s = (ice_day_ds.LAKEICETHICK[:,:3650,:,:] > 0).groupby('time.month').sum('time', skipna=True).mean('ensemble').compute().where(~np.isnan(ice_day_ds.LAKEICETHICK[0,0,...].drop(('ensemble','time')).compute())) / 10
ice_mon_climmean_1850s = ice_mon_climmean_1850s.where(~np.isnan(overturn_jconv_days_mon_1850s[0,...].drop('month')))

# now calculate the phenology change under different warming level

In [ ]:
print(u"year with global temperature mean anomayly of 1.5 \u2103: " + str(trefht_globmean.year[np.fabs(trefht_globmean.Temperature_glob_mean - trefht_globmean.Temperature_glob_mean[:50].mean() -1.5).argmin()]))
print(u"year with global temperature mean anomayly of 3 \u2103: " + str(trefht_globmean.year[np.fabs(trefht_globmean.Temperature_glob_mean - trefht_globmean.Temperature_glob_mean[:50].mean() -3).argmin()]))
print(u"year with global temperature mean anomayly of 4.5 \u2103: " + str(trefht_globmean.year[np.fabs(trefht_globmean.Temperature_glob_mean - trefht_globmean.Temperature_glob_mean[:50].mean() -4.5).argmin()]))

year with global temperature mean anomayly of 1.5 ℃: 2029.0
year with global temperature mean anomayly of 3 ℃: 2067.0
year with global temperature mean anomayly of 4.5 ℃: 2096.0


In [ ]:
strat_days_1850_lunit = (xr.apply_ufunc(lambda x: x.max() - x[x > 0].min(),
              tlake_day_ds.TLAKE[:,:365,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
strat_days_2029_lunit = (xr.apply_ufunc(lambda x: x.max() - x[x > 0].min(),
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2029-01-01','2029-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
strat_days_2067_lunit = (xr.apply_ufunc(lambda x: x.max() - x[x > 0].min(),
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2067-01-01','2067-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
strat_days_2096_lunit = (xr.apply_ufunc(lambda x: x.max() - x[x > 0].min(),
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2096-01-01','2096-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()

In [ ]:
inverstrat_days_1850_lunit = (xr.apply_ufunc(lambda x: x[-1] - x[x>0][0],
              tlake_day_ds.TLAKE[:,:365,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
inverstrat_days_2029_lunit = (xr.apply_ufunc(lambda x: x[-1] - x[x>0][0],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2029-01-01','2029-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
inverstrat_days_2067_lunit = (xr.apply_ufunc(lambda x: x[-1] - x[x>0][0],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2067-01-01','2067-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
inverstrat_days_2096_lunit = (xr.apply_ufunc(lambda x: x[-1] - x[x>0][0],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2096-01-01','2096-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()

In [ ]:
positstrat_days_1850_lunit = (xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:365,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
positstrat_days_2029_lunit = (xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2029-01-01','2029-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
positstrat_days_2067_lunit = (xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2067-01-01','2067-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()
positstrat_days_2096_lunit = (xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2096-01-01','2096-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']) > 1).groupby('time.month').sum('time').compute()


In [11]:
overturn_jconv_days_1850 = (jconv_day_ds.JCONVECT[:,:365,:,:] > 8).groupby('time.month').sum('time').mean('ensemble').compute().where(~np.isnan(jconv_day_ds.JCONVECT[0,0,:,:].load().drop(('ensemble','time')))).where(~np.isnan(strat_days_1850s_ensmean))
overturn_jconv_days_2029 = (jconv_day_ds.JCONVECT[:,:,:,:].sel(time = slice('2029-01-01','2029-12-31')) > 8).groupby('time.month').sum('time').mean('ensemble').compute().where(~np.isnan(jconv_day_ds.JCONVECT[0,0,:,:].load().drop(('ensemble','time')))).where(~np.isnan(strat_days_1850s_ensmean))
overturn_jconv_days_2067 = (jconv_day_ds.JCONVECT[:,:,:,:].sel(time = slice('2067-01-01','2067-12-31')) > 8).groupby('time.month').sum('time').mean('ensemble').compute().where(~np.isnan(jconv_day_ds.JCONVECT[0,0,:,:].load().drop(('ensemble','time')))).where(~np.isnan(strat_days_1850s_ensmean))
overturn_jconv_days_2096 = (jconv_day_ds.JCONVECT[:,:,:,:].sel(time = slice('2096-01-01','2096-12-31')) > 8).groupby('time.month').sum('time').mean('ensemble').compute().where(~np.isnan(jconv_day_ds.JCONVECT[0,0,:,:].load().drop(('ensemble','time')))).where(~np.isnan(strat_days_1850s_ensmean))

In [28]:

strat_days_changes_2029_lunit = (strat_days_2029_lunit - strat_days_1850_lunit).mean('ensemble')
strat_days_changes_2067_lunit = (strat_days_2067_lunit - strat_days_1850_lunit).mean('ensemble')
strat_days_changes_2096_lunit = (strat_days_2096_lunit - strat_days_1850_lunit).mean('ensemble')
inverstrat_days_changes_2029_lunit = (inverstrat_days_2029_lunit - inverstrat_days_1850_lunit).mean('ensemble')
inverstrat_days_changes_2067_lunit = (inverstrat_days_2067_lunit - inverstrat_days_1850_lunit).mean('ensemble')
inverstrat_days_changes_2096_lunit = (inverstrat_days_2096_lunit - inverstrat_days_1850_lunit).mean('ensemble')
positstrat_days_changes_2029_lunit = (positstrat_days_2029_lunit - positstrat_days_1850_lunit).mean('ensemble')
positstrat_days_changes_2067_lunit = (positstrat_days_2067_lunit - positstrat_days_1850_lunit).mean('ensemble')
positstrat_days_changes_2096_lunit = (positstrat_days_2096_lunit - positstrat_days_1850_lunit).mean('ensemble')

overturn_jconv_days_changes_2029 = overturn_jconv_days_2029 - overturn_jconv_days_1850
overturn_jconv_days_changes_2067 = overturn_jconv_days_2067 - overturn_jconv_days_1850
overturn_jconv_days_changes_2096 = overturn_jconv_days_2096 - overturn_jconv_days_1850


In [58]:
strat_days_changes_2029 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    strat_days_changes_2029[:, lake_jxy[i], lake_ixy[i]] = strat_days_changes_2029_lunit[:,i].values
strat_days_changes_2029 = xr.DataArray(
    strat_days_changes_2029,
    dims=('month', 'lat', 'lon'),
    coords={'month': strat_days_changes_2029_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of stratification (sum of positive and inverse stratification)',
           'units': 'days'},
    name='STRAT_CHANGE'
)

strat_days_changes_2067 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    strat_days_changes_2067[:, lake_jxy[i], lake_ixy[i]] = strat_days_changes_2067_lunit[:,i].values
strat_days_changes_2067 = xr.DataArray(
    strat_days_changes_2067,
    dims=('month', 'lat', 'lon'),
    coords={'month': strat_days_changes_2067_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of stratification (sum of positive and inverse stratification)',
           'units': 'days'},
    name='STRAT_CHANGE'
)


strat_days_changes_2096 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    strat_days_changes_2096[:, lake_jxy[i], lake_ixy[i]] = strat_days_changes_2096_lunit[:,i].values
strat_days_changes_2096 = xr.DataArray(
    strat_days_changes_2096,
    dims=('month', 'lat', 'lon'),
    coords={'month': strat_days_changes_2096_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of stratification (sum of positive and inverse stratification)',
           'units': 'days'},
    name='STRAT_CHANGE'
)

inverstrat_days_changes_2029 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    inverstrat_days_changes_2029[:, lake_jxy[i], lake_ixy[i]] = inverstrat_days_changes_2029_lunit[:,i].values
inverstrat_days_changes_2029 = xr.DataArray(
    inverstrat_days_changes_2029,
    dims=('month', 'lat', 'lon'),
    coords={'month': inverstrat_days_changes_2029_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of inverse stratification',
           'units': 'days'},
    name='INVERSTRAT_CHANGE'
)

inverstrat_days_changes_2067 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    inverstrat_days_changes_2067[:, lake_jxy[i], lake_ixy[i]] = inverstrat_days_changes_2067_lunit[:,i].values
inverstrat_days_changes_2067 = xr.DataArray(
    inverstrat_days_changes_2067,
    dims=('month', 'lat', 'lon'),
    coords={'month': inverstrat_days_changes_2067_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of inverse stratification',
           'units': 'days'},
    name='INVERSTRAT_CHANGE'
)

inverstrat_days_changes_2096 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    inverstrat_days_changes_2096[:, lake_jxy[i], lake_ixy[i]] = inverstrat_days_changes_2096_lunit[:,i].values
inverstrat_days_changes_2096 = xr.DataArray(
    inverstrat_days_changes_2096,
    dims=('month', 'lat', 'lon'),
    coords={'month': inverstrat_days_changes_2096_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of inverse stratification',
           'units': 'days'},
    name='INVERSTRAT_CHANGE'
)

positstrat_days_changes_2029 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    positstrat_days_changes_2029[:, lake_jxy[i], lake_ixy[i]] = positstrat_days_changes_2029_lunit[:,i].values
positstrat_days_changes_2029 = xr.DataArray(
    positstrat_days_changes_2029,
    dims=('month', 'lat', 'lon'),
    coords={'month': positstrat_days_changes_2029_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_CHANGE'
)

positstrat_days_changes_2067 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    positstrat_days_changes_2067[:, lake_jxy[i], lake_ixy[i]] = positstrat_days_changes_2067_lunit[:,i].values
positstrat_days_changes_2067 = xr.DataArray(
    positstrat_days_changes_2067,
    dims=('month', 'lat', 'lon'),
    coords={'month': positstrat_days_changes_2067_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_CHANGE'
)

positstrat_days_changes_2096 = np.empty((12,192,288)) * np.nan
for i in np.arange(4622):
    positstrat_days_changes_2096[:, lake_jxy[i], lake_ixy[i]] = positstrat_days_changes_2096_lunit[:,i].values
positstrat_days_changes_2096 = xr.DataArray(
    positstrat_days_changes_2096,
    dims=('month', 'lat', 'lon'),
    coords={'month': positstrat_days_changes_2096_lunit.month, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in duration of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_CHANGE'
)


In [ ]:
ice_days_changes = ((ice_day_ds.LAKEICETHICK.sel(time = slice('2096-01-01','2096-12-31')) > 0 ).groupby('time.month').sum('time')\
    -(ice_day_ds.LAKEICETHICK[:,:365,:,:] > 0 ).groupby('time.month').sum('time') ).mean('ensemble').compute().where(~np.isnan(ice_day_ds.LAKEICETHICK[0,0,:,:].load().drop(('ensemble','time')))).where(~np.isnan(strat_days_1850s_ensmean))

## calculate changes in momentum flux under different warming level

In [29]:
ice_day_lunit = ice_day_ds.LAKEICETHICK[:,:,lake_jxy, lake_ixy]
u10_day_lunit = u10_day_ds.U10[:,:,lake_jxy,lake_ixy]
u10_day_lunit_icemasked = xr.where(ice_day_lunit > 0, 0, u10_day_lunit)

2024-11-21 12:59:37,117 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
u10_mon_lunit_diff_icemasked = ((u10_day_lunit_icemasked.sel(time = slice('2096-01-01','2096-12-31')).groupby('time.month').mean('time') \
                                        - u10_day_lunit_icemasked[:,:365,:].groupby('time.month').mean('time'))).mean('ensemble').compute()    
                                        
u10_mon_diff_icemasked = np.empty((12,192,288)) * np.nan
for i in np.arange(u10_mon_lunit_diff_icemasked.landunit.size):
    u10_mon_diff_icemasked[:, lake_jxy[i], lake_ixy[i]] = u10_mon_lunit_diff_icemasked[:,i].values
u10_mon_diff_icemasked = xr.DataArray(
                                 data = u10_mon_diff_icemasked,
                                 dims = ('month','lat','lon'),
                                 coords = {'month': np.arange(1,13),
                                          'lat':u10_mon_ds.lat,
                                          'lon':u10_mon_ds.lon},
                                name = 'U10'
                                    ) 

# write the ensemble mean results to the netcdf file

In [ ]:
strat_days_1850s_ensmean.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically annual mean of total \
stratification (additive sum of summer stratification and inverse stratification)',
    'calculation': 'number of days with temperature difference between surface layer and bottom layer larger than 1 degree celcius, based on the daily lake temperature data (TLAKE) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Annual_Total_Stratification_EnsembleMean_1850.nc')

inverstrat_days_1850s_ensmean.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically annual mean of inverse \
stratification',
    'calculation': 'number of days with bottom layer temperature 1 degree celcius warmer than the surface of unfrozen layers, based on the daily lake temperature data (TLAKE) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Annual_Inverse_Stratification_EnsembleMean_1850.nc')

positstrat_days_1850s_ensmean.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically annual mean of summer \
stratification',
    'calculation': 'number of days with surface of unfrozen layers 1 degree celcius warmer than the bottom layer , based on the daily lake temperature data (TLAKE) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Annual_Summer_Stratification_EnsembleMean_1850.nc')

strat_days_1850s_mon_ensmean.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically monthly mean duration of total \
stratification (additive sum of summer stratification and inverse stratification)',
    'calculation': 'number of days with temperature difference between surface layer and bottom layer larger than 1 degree celcius, based on the daily lake temperature data (TLAKE) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Total_Stratification_EnsembleMean_1850s.nc')

inverstrat_days_1850s_mon_ensmean.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically monthly mean duration of inverse \
stratification',
    'calculation': 'number of days with bottom layer temperature 1 degree celcius warmer than the surface of unfrozen layers, based on the daily lake temperature data (TLAKE) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Inverse_Stratification_EnsembleMean_1850s.nc')

positstrat_days_1850s_mon_ensmean.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically monthly mean duration of summer \
stratification',
    'calculation': 'number of days with surface of unfrozen layers 1 degree celcius warmer than the bottom layer , based on the daily lake temperature data (TLAKE) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Summer_Stratification_EnsembleMean_1850s.nc')

overturn_jconv_mon_1850s.rename('OVERTURN').assign_attrs({'long_name': 'monthly duration of overturning', 'units':'days'}).to_dataset().assign_attrs(
    {'attribution':'this is the ensemble mean of climatologically monthly mean of overturning duration',
     'calculation': 'number of days with at least one full-column convection event occuring, based on the daily maximum of covection layers (JCONVECT) from the CESM2-LE',
     'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Overturning_EnsembleMean_1850s.nc')

tsa_mon_climmean_1850s.to_dataset().assign_attrs({'attribution': 'this is the ensemble mean of climatologically monthly mean land surface air temperature over lake grids',
    'calculation': 'based on the daily land surface air tempeature (TSA) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_LandSurfaceAirTemperature_EnsembleMean_1850s.nc')

ice_mon_climmean_1850s = xr.DataArray(
    ice_mon_climmean_1850s.values,
    dims=('month','lat', 'lon'),
    coords={'month':np.arange(1,13),'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'monthly duration of lake ice cover',
           'units': 'days'},
    name='ICEDURATION'
)
ice_mon_climmean_1850s.to_dataset().assign_attrs(
    {'attribution': 'this is the ensemble mean of climatologically monthly mean duration of lake ice cover',
    'calculation': 'based on the daily lake ice thickness data (LAKEICETHICK) from the CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'}
).to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_LakeIceCover_EnsembleMean_1850s.nc')


In [216]:
strat_days_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of total stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Total_Stratification_EnsembleMean_2029-1850.nc')

strat_days_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of total stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Total_Stratification_EnsembleMean_2067-1850.nc')

strat_days_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of total stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Total_Stratification_EnsembleMean_2096-1850.nc')


inverstrat_days_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of invserse stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Inverse_Stratification_EnsembleMean_2029-1850.nc')

inverstrat_days_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of invserse stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Inverse_Stratification_EnsembleMean_2067-1850.nc')

inverstrat_days_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of invserse stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Inverse_Stratification_EnsembleMean_2096-1850.nc')


positstrat_days_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of summer stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Summer_Stratification_EnsembleMean_2029-1850.nc')

positstrat_days_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of summer stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Summer_Stratification_EnsembleMean_2067-1850.nc')

positstrat_days_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of summer stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Summer_Stratification_EnsembleMean_2096-1850.nc')

overturn_jconv_days_changes_2029.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in duration of overturning', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of overturning',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Overturning_EnsembleMean_2029-1850.nc')
overturn_jconv_days_changes_2067.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in duration of overturning', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of overturning',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Overturning_EnsembleMean_2067-1850.nc')
overturn_jconv_days_changes_2096.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in duration of overturning', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of overturning',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_Overturning_EnsembleMean_2096-1850.nc')

ice_days_changes.rename('ICEDU_CHANGE').assign_attrs({'long_name': 'changes in duration of lake ice cover', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in monthly mean duration of lake ice cover',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_LakeIceCover_EnsembleMean_2096-1850.nc')


In [ ]:
u10_mon_diff_icemasked.rename('U10_CHANGE').assign_attrs({'long_name': 'changes in monthly mean surface wind (10 m height) over lake grids', 'units': 'm/s'})\
    .to_dataset().assign_attrs({'attribution': 'changes in monthly mean surface wind',
    'calculation': '2096 - 1850, surface wind is set to 0 when ice cover thicker than 3 cm, monthly data are calculated based on daily surface wind data (U10) from CESM2-LE',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Monthly_U10_EnsembleMean_2096-1850.nc')


# calculate the start and end of summer stratification/overturning

Summer stratificaiton

In [42]:
temp_diff_1850_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:365,:,lake_jxy,lake_ixy].chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})
temp_diff_2029_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2029-01-01','2029-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})

temp_diff_2067_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2067-01-01','2067-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})
              
temp_diff_2096_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2096-01-01','2096-12-31')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})
              

In [43]:
positstrat_days_1850_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_1850_lunit > 1)
positstrat_days_2029_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_2029_lunit > 1)
positstrat_days_2067_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_2067_lunit > 1)
positstrat_days_2096_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_2096_lunit > 1)

In [44]:
positstrat_days_1850_lunit_start = positstrat_days_1850_lunit_mask.min('dayofyear').compute()
positstrat_days_2029_lunit_start = positstrat_days_2029_lunit_mask.min('dayofyear').compute()
positstrat_days_2067_lunit_start = positstrat_days_2067_lunit_mask.min('dayofyear').compute()
positstrat_days_2096_lunit_start = positstrat_days_2096_lunit_mask.min('dayofyear').compute()
positstrat_days_1850_lunit_end = positstrat_days_1850_lunit_mask.max('dayofyear').compute()
positstrat_days_2029_lunit_end = positstrat_days_2029_lunit_mask.max('dayofyear').compute()
positstrat_days_2067_lunit_end = positstrat_days_2067_lunit_mask.max('dayofyear').compute()
positstrat_days_2096_lunit_end = positstrat_days_2096_lunit_mask.max('dayofyear').compute()

In [45]:
positstrat_days_1850_start = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_1850_start[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_1850_lunit_start[:,i].values
positstrat_days_1850_start = xr.DataArray(
    positstrat_days_1850_start,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_1850_lunit_start.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'start day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_START'
)
positstrat_days_2029_start = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_2029_start[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_2029_lunit_start[:,i].values
positstrat_days_2029_start = xr.DataArray(
    positstrat_days_2029_start,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_2029_lunit_start.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'start day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_START'
)
positstrat_days_2067_start = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_2067_start[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_2067_lunit_start[:,i].values
positstrat_days_2067_start = xr.DataArray(
    positstrat_days_2067_start,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_2067_lunit_start.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'start day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_START'
)
positstrat_days_2096_start = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_2096_start[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_2096_lunit_start[:,i].values
positstrat_days_2096_start = xr.DataArray(
    positstrat_days_2096_start,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_2096_lunit_start.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'start day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_START'
)

In [46]:
positstrat_days_1850_end = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_1850_end[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_1850_lunit_end[:,i].values
positstrat_days_1850_end = xr.DataArray(
    positstrat_days_1850_end,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_1850_lunit_end.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'end day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_END'
)
positstrat_days_2029_end = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_2029_end[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_2029_lunit_end[:,i].values
positstrat_days_2029_end = xr.DataArray(
    positstrat_days_2029_end,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_2029_lunit_end.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'end day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_END'
)
positstrat_days_2067_end = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_2067_end[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_2067_lunit_end[:,i].values
positstrat_days_2067_end = xr.DataArray(
    positstrat_days_2067_end,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_2067_lunit_end.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'end day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_END'
)
positstrat_days_2096_end = np.empty((90, 192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    positstrat_days_2096_end[:,lake_jxy[i], lake_ixy[i]] = positstrat_days_2096_lunit_end[:,i].values
positstrat_days_2096_end = xr.DataArray(
    positstrat_days_2096_end,
    dims=('ensemble', 'lat', 'lon'),
    coords={'ensemble': positstrat_days_2096_lunit_end.ensemble, 
            'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'end day of positive stratification',
           'units': 'days'},
    name='POSITSTRAT_END'
)

In [47]:
del positstrat_days_1850_lunit_start ,positstrat_days_2029_lunit_start ,positstrat_days_2067_lunit_start ,positstrat_days_2096_lunit_start
del positstrat_days_1850_lunit_end ,positstrat_days_2029_lunit_end ,positstrat_days_2067_lunit_end ,positstrat_days_2096_lunit_end
gc.collect()

206

In [50]:
positstrat_start_changes_2029 = (positstrat_days_2029_start - positstrat_days_1850_start).mean('ensemble').where(ice_ann_climmean_1850s > 0)
positstrat_start_changes_2067 = (positstrat_days_2067_start - positstrat_days_1850_start).mean('ensemble').where(ice_ann_climmean_1850s > 0)
positstrat_start_changes_2096 = (positstrat_days_2096_start - positstrat_days_1850_start).mean('ensemble').where(ice_ann_climmean_1850s > 0)

positstrat_end_changes_2029 = (positstrat_days_2029_end - positstrat_days_1850_end).mean('ensemble').where(ice_ann_climmean_1850s > 0)
positstrat_end_changes_2067 = (positstrat_days_2067_end - positstrat_days_1850_end).mean('ensemble').where(ice_ann_climmean_1850s > 0)
positstrat_end_changes_2096 = (positstrat_days_2096_end - positstrat_days_1850_end).mean('ensemble').where(ice_ann_climmean_1850s > 0)

Inverse Stratification

In [51]:
temp_diff_1850_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('1850-07-15','1851-07-14')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})
temp_diff_2029_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2029-07-15','2030-07-14')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})

temp_diff_2067_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2067-07-15','2068-07-14')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})
              
temp_diff_2096_lunit = xr.apply_ufunc(lambda x: x[x>0][0] - x[-1],
              tlake_day_ds.TLAKE[:,:,:,lake_jxy,lake_ixy].sel(time = slice('2096-07-15','2097-07-14')).chunk({'ensemble':1,'landunit':500}),
              input_core_dims=[['levlak'],],
             exclude_dims=set(('levlak',)),
              vectorize = True,
              dask='parallelized',
              output_dtypes=['float32']).rename({'time':'dayofyear'}).assign_coords({'dayofyear':np.arange(1,366)})
              

In [52]:
inverstrat_days_1850_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_1850_lunit < -1)
inverstrat_days_2029_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_2029_lunit < -1)
inverstrat_days_2067_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_2067_lunit < -1)
inverstrat_days_2096_lunit_mask = temp_diff_1850_lunit.dayofyear.where(temp_diff_2096_lunit < -1)


inverstrat_start_changes_2029_lunit =  (inverstrat_days_2029_lunit_mask.min('dayofyear') - inverstrat_days_1850_lunit_mask.min('dayofyear')).mean('ensemble').compute()
inverstrat_start_changes_2067_lunit =  (inverstrat_days_2067_lunit_mask.min('dayofyear') - inverstrat_days_1850_lunit_mask.min('dayofyear')).mean('ensemble').compute()
inverstrat_start_changes_2096_lunit =  (inverstrat_days_2096_lunit_mask.min('dayofyear') - inverstrat_days_1850_lunit_mask.min('dayofyear')).mean('ensemble').compute()
inverstrat_end_changes_2029_lunit =  (inverstrat_days_2029_lunit_mask.max('dayofyear') - inverstrat_days_1850_lunit_mask.max('dayofyear')).mean('ensemble').compute()
inverstrat_end_changes_2067_lunit =  (inverstrat_days_2067_lunit_mask.max('dayofyear') - inverstrat_days_1850_lunit_mask.max('dayofyear')).mean('ensemble').compute()
inverstrat_end_changes_2096_lunit =  (inverstrat_days_2096_lunit_mask.max('dayofyear') - inverstrat_days_1850_lunit_mask.max('dayofyear')).mean('ensemble').compute()


/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using future

In [53]:
inverstrat_start_changes_2029 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    inverstrat_start_changes_2029[lake_jxy[i], lake_ixy[i]] = inverstrat_start_changes_2029_lunit[i].values
inverstrat_start_changes_2029 = xr.DataArray(
    inverstrat_start_changes_2029,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in start day of inverse stratification',
           'units': 'days'},
    name='INVERSTART_START'
).where(ice_ann_climmean_1850s > 0)

inverstrat_start_changes_2067 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    inverstrat_start_changes_2067[lake_jxy[i], lake_ixy[i]] = inverstrat_start_changes_2067_lunit[i].values
inverstrat_start_changes_2067 = xr.DataArray(
    inverstrat_start_changes_2067,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in start day of inverse stratification',
           'units': 'days'},
    name='INVERSTART_START'
).where(ice_ann_climmean_1850s > 0)

inverstrat_start_changes_2096 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    inverstrat_start_changes_2096[lake_jxy[i], lake_ixy[i]] = inverstrat_start_changes_2096_lunit[i].values
inverstrat_start_changes_2096 = xr.DataArray(
    inverstrat_start_changes_2096,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in start day of inverse stratification',
           'units': 'days'},
    name='INVERSTART_START'
).where(ice_ann_climmean_1850s > 0)

inverstrat_end_changes_2029 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    inverstrat_end_changes_2029[lake_jxy[i], lake_ixy[i]] = inverstrat_end_changes_2029_lunit[i].values
inverstrat_end_changes_2029 = xr.DataArray(
    inverstrat_end_changes_2029,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in end day of inverse stratification',
           'units': 'days'},
    name='INVERSTRAT_END'
).where(ice_ann_climmean_1850s > 0)

inverstrat_end_changes_2067 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    inverstrat_end_changes_2067[lake_jxy[i], lake_ixy[i]] = inverstrat_end_changes_2067_lunit[i].values
inverstrat_end_changes_2067 = xr.DataArray(
    inverstrat_end_changes_2067,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in end day of inverse stratification',
           'units': 'days'},
    name='INVERSTRAT_END'
).where(ice_ann_climmean_1850s > 0)

inverstrat_end_changes_2096 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    inverstrat_end_changes_2096[lake_jxy[i], lake_ixy[i]] = inverstrat_end_changes_2096_lunit[i].values
inverstrat_end_changes_2096 = xr.DataArray(
    inverstrat_end_changes_2096,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in end day of inverse stratification',
           'units': 'days'},
    name='INVERSTRAT_END'
).where(ice_ann_climmean_1850s > 0)

/tmp/ipykernel_19300/4243654.py:49: RuntimeWarning: invalid value encountered in multiply
  inverstrat_end_changes_2067 = np.empty((192,288)) * np.NaN


overturning duration in spring and winter, respectively

In [54]:
overturn_spring_duration_1850_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('1850-04-01','1850-09-30'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')
overturn_spring_duration_2029_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('2029-04-01','2029-09-30'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')
overturn_spring_duration_2067_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('2067-04-01','2067-09-30'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')
overturn_spring_duration_2096_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('2096-04-01','2096-09-30'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')

overturn_spring_duration_changes_2029_lunit = (overturn_spring_duration_2029_lunit - overturn_spring_duration_1850_lunit).mean('ensemble').compute()
overturn_spring_duration_changes_2067_lunit = (overturn_spring_duration_2067_lunit - overturn_spring_duration_1850_lunit).mean('ensemble').compute()
overturn_spring_duration_changes_2096_lunit = (overturn_spring_duration_2096_lunit - overturn_spring_duration_1850_lunit).mean('ensemble').compute()

overturn_spring_duration_changes_2029 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    overturn_spring_duration_changes_2029[lake_jxy[i], lake_ixy[i]] = overturn_spring_duration_changes_2029_lunit[i].values
overturn_spring_duration_changes_2029 = xr.DataArray(
    overturn_spring_duration_changes_2029,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in spring duration (day) of overturning',
           'units': 'days'},
    name='overturn_spring_duration'
).where(ice_ann_climmean_1850s > 0)

overturn_spring_duration_changes_2067 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    overturn_spring_duration_changes_2067[lake_jxy[i], lake_ixy[i]] = overturn_spring_duration_changes_2067_lunit[i].values
overturn_spring_duration_changes_2067 = xr.DataArray(
    overturn_spring_duration_changes_2067,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in spring duration (day) of overturning',
           'units': 'days'},
    name='overturn_spring_duration'
).where(ice_ann_climmean_1850s > 0)

overturn_spring_duration_changes_2096 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    overturn_spring_duration_changes_2096[lake_jxy[i], lake_ixy[i]] = overturn_spring_duration_changes_2096_lunit[i].values
overturn_spring_duration_changes_2096 = xr.DataArray(
    overturn_spring_duration_changes_2096,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in spring duration (day) of overturning',
           'units': 'days'},
    name='overturn_spring_duration'
).where(ice_ann_climmean_1850s > 0)

In [55]:
overturn_winter_duration_1850_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('1850-10-01','1851-03-31'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')
overturn_winter_duration_2029_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('2029-10-01','2030-03-31'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')
overturn_winter_duration_2067_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('2067-10-01','2068-03-31'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')
overturn_winter_duration_2096_lunit = (jconv_day_ds.JCONVECT[:,:,lake_jxy,lake_ixy].sel(time = slice('2096-10-01','2097-03-31'))\
    .chunk({'ensemble':1, 'landunit':500}) > 8 ).sum('time')

overturn_winter_duration_changes_2029_lunit = (overturn_winter_duration_2029_lunit - overturn_winter_duration_1850_lunit).mean('ensemble').compute()
overturn_winter_duration_changes_2067_lunit = (overturn_winter_duration_2067_lunit - overturn_winter_duration_1850_lunit).mean('ensemble').compute()
overturn_winter_duration_changes_2096_lunit = (overturn_winter_duration_2096_lunit - overturn_winter_duration_1850_lunit).mean('ensemble').compute()

overturn_winter_duration_changes_2029 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    overturn_winter_duration_changes_2029[lake_jxy[i], lake_ixy[i]] = overturn_winter_duration_changes_2029_lunit[i].values
overturn_winter_duration_changes_2029 = xr.DataArray(
    overturn_winter_duration_changes_2029,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in winter duration (day) of overturning',
           'units': 'days'},
    name='overturn_winter_duration'
).where(ice_ann_climmean_1850s > 0)

overturn_winter_duration_changes_2067 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    overturn_winter_duration_changes_2067[lake_jxy[i], lake_ixy[i]] = overturn_winter_duration_changes_2067_lunit[i].values
overturn_winter_duration_changes_2067 = xr.DataArray(
    overturn_winter_duration_changes_2067,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in winter duration (day) of overturning',
           'units': 'days'},
    name='overturn_winter_duration'
).where(ice_ann_climmean_1850s > 0)

overturn_winter_duration_changes_2096 = np.empty((192,288)) * np.NaN
for i in np.arange(lake_ixy.size):
    overturn_winter_duration_changes_2096[lake_jxy[i], lake_ixy[i]] = overturn_winter_duration_changes_2096_lunit[i].values
overturn_winter_duration_changes_2096 = xr.DataArray(
    overturn_winter_duration_changes_2096,
    dims=('lat', 'lon'),
    coords={'lat': tlake_day_ds.lat, 'lon': tlake_day_ds.lon},
    attrs={'long_name': 'changes in winter duration (day) of overturning',
           'units': 'days'},
    name='overturn_winter_duration'
).where(ice_ann_climmean_1850s > 0)

/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/huanglei/usr/anaconda3/envs/Jun2023/lib/python3.9/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 12.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [59]:
overturn_winter_duration_changes_2029.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in overturning duration in winter', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in overturning duration in winter',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Winter_Overturning_EnsembleMean_2029-1850.nc')

overturn_winter_duration_changes_2067.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in overturning duration in winter', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in overturning duration in winter',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Winter_Overturning_EnsembleMean_2067-1850.nc')

overturn_winter_duration_changes_2096.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in overturning duration in winter', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in overturning duration in winter',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Winter_Overturning_EnsembleMean_2096-1850.nc')

overturn_spring_duration_changes_2029.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in overturning duration in spring', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in overturning duration in spring',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Spring_Overturning_EnsembleMean_2029-1850.nc')

overturn_spring_duration_changes_2067.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in overturning duration in spring', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in overturning duration in spring',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Spring_Overturning_EnsembleMean_2067-1850.nc')

overturn_spring_duration_changes_2096.rename('OVERTURN_CHANGE').assign_attrs({'long_name': 'changes in overturning duration in spring', 'units': 'days'})\
    .to_dataset().assign_attrs({'attribution': 'changes in overturning duration in spring',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Spring_Overturning_EnsembleMean_2096-1850.nc')


In [64]:
inverstrat_start_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in start of invserse stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Start_Inverse_Stratification_EnsembleMean_2029-1850.nc')

inverstrat_start_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in start of invserse stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Start_Inverse_Stratification_EnsembleMean_2067-1850.nc')

inverstrat_start_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in start of invserse stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Start_Inverse_Stratification_EnsembleMean_2096-1850.nc')

inverstrat_end_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in end of invserse stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/End_Inverse_Stratification_EnsembleMean_2029-1850.nc')

inverstrat_end_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in end of invserse stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/End_Inverse_Stratification_EnsembleMean_2067-1850.nc')

inverstrat_end_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in end of invserse stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/End_Inverse_Stratification_EnsembleMean_2096-1850.nc')



In [66]:
positstrat_start_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in start of summer stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Start_Summer_Stratification_EnsembleMean_2029-1850.nc')

positstrat_start_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in start of summer stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Start_Summer_Stratification_EnsembleMean_2067-1850.nc')

positstrat_start_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in start of summer stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/Start_Summer_Stratification_EnsembleMean_2096-1850.nc')

positstrat_end_changes_2029.to_dataset().assign_attrs({'attribution': 'changes in end of summer stratification',
    'calculation': '2029 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/End_Summer_Stratification_EnsembleMean_2029-1850.nc')

positstrat_end_changes_2067.to_dataset().assign_attrs({'attribution': 'changes in end of summer stratification',
    'calculation': '2067 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/End_Summer_Stratification_EnsembleMean_2067-1850.nc')

positstrat_end_changes_2096.to_dataset().assign_attrs({'attribution': 'changes in end of summer stratification',
    'calculation': '2096 - 1850',
    'history':'created by Lei Huang, Nov 28th, 2023'})\
        .to_netcdf('/proj/lhuang/scripts/CESM2-LENS/Lake_Mixing_phenology/NCs/End_Summer_Stratification_EnsembleMean_2096-1850.nc')

